# Natural Language Processing - Group 1
**Member:**

Trần Văn Đức - 20020077

Nguyễn Tấn Minh - 20020081

Nguyễn Hoàng Trung - 20020083

# Import library

In [1]:
import re
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

# Pre-process

In [2]:
teencode = open("/kaggle/input/sa-vi/teencode.txt", "r").read().split('\n')
for i in range(len(teencode)):
    teencode[i] = teencode[i].split("\t")

In [3]:
def remove_url(text):
    text = re.sub(r'(http\S+)?(\w+\.)+\S+', r'', text)
    return text

remove_url('Vn cũng cuồng www.example.com aw chả kém gì http://genk.vn/do-choi-so/nguoi-vie...ang-la-tinh-trang-chung-20150509171244829.chn Người Việt cũng cuồng Apple Watch, khan hàng là tình trạng chung Mặc cho những dự đoán không mấy khả quan về doanh số của mình, Apple Watch đã liên tục rơi vào tình trạng cháy hàng trên toàn thế giới. genk.vn  ')

'Vn cũng cuồng  aw chả kém gì  Người Việt cũng cuồng Apple Watch, khan hàng là tình trạng chung Mặc cho những dự đoán không mấy khả quan về doanh số của mình, Apple Watch đã liên tục rơi vào tình trạng cháy hàng trên toàn thế giới.  \xa0'

In [4]:
def convert_upper_to_lower(text):
    return text.lower()

convert_upper_to_lower('Máy này mà render phim HD dựng bằng Premiere, After Effect... chắc sướng phải biết! Mơ 1 cái!')

'máy này mà render phim hd dựng bằng premiere, after effect... chắc sướng phải biết! mơ 1 cái!'

In [5]:
def remove_special_character(text):
    text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ]', r' ', text)
    return text

remove_special_character('màn hình lên đến 18.4 inh + cấu hình khủng như vậy thì quá ổn rồi,,,quá hợp với gamer ..... đúng là nhà thiết kế hiểu được a/e gamer.... thanhk....')

'màn hình lên đến 18 4 inh   cấu hình khủng như vậy thì quá ổn rồi   quá hợp với gamer       đúng là nhà thiết kế hiểu được a e gamer     thanhk    '

In [6]:
def remove_duplicate_character(text):
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    text = re.sub(r'(\s)\1+', r'\1', text)
    return text

remove_duplicate_character('niceeeeeee  facebook   !!!!!!!!')

'nice facebook !'

In [7]:
def remove_acronyms(text):
    for x in teencode: 
        text = re.sub(fr' {x[0]} |^{x[0]} | {x[0]}$', fr' {x[1]} ', text)
    return text

remove_acronyms('thật k thể tin nổi, ko có từ j để diễn tả')

'thật không thể tin nổi, không có từ gì để diễn tả'

In [8]:
def truncate(text):
    text = text.split()
    text = text[:900]
    return " ".join(text)

In [9]:
def pre_process(data):
    for i, x in enumerate(data):
        data[i] = remove_url(data[i])
        data[i] = convert_upper_to_lower(data[i])
        data[i] = remove_special_character(data[i])
        data[i] = remove_duplicate_character(data[i])
        data[i] = remove_acronyms(data[i])
#         data[i] = truncate(data[i])
    return data

pre_process(['Thật tiếc là đa số những ứng dụng trên đều không sử dụng được hoặc chưa thật sự hiểu quả tại Việt Nam cơ bản thì mình cũng có cùng quan điểm với bác chủ topic về khái niệm sử dụng smartwatch vào thời điểm hiện tại.Trung bình 300-400$ là 1 khoản tiền k nhỏ,kiếm 300-400$ tại Việt Nam cũng k phải đơn giản nhưng việc 1 người bỏ ra 300-400$ để thoả mãn đam mê của mình thì ngay cả ở Việt Nam cũng đâu phải cái j đó quá khủng khiếp.Mình thấy lạ là có rất nhiều người vào comment cái kiểu "chả để làm j", "nâng bi", "phí tiền",...Mình k nói bạn có tiền để mua hay k,nhưng đơn giản là bạn vào đọc,k thk bạn có thể đi ra và tìm 1 topic khác đúng sở thk của bạn.Bạn k cần phải thở ra những câu chả liên quan kiểu như vậy để người khác k mua giống mình cho đỡ cảm thấy lạc lõng vs xu hướng của thế giới   '])

['thật tiếc là đa số những ứng dụng trên đều không sử dụng được hoặc chưa thật sự hiểu quả tại việt nam cơ bản thì mình cũng có cùng quan điểm với bác chủ topic về khái niệm sử dụng smartwatch vào thời điểm hiện bình 300 400 là 1 khoản tiền không nhỏ kiếm 300 400 tại việt nam cũng không phải đơn giản nhưng việc 1 người bỏ ra 300 400 để thoả mãn đam mê của mình thì ngay cả ở việt nam cũng đâu phải cái gì đó quá khủng thấy lạ là có rất nhiều người vào comment cái kiểu chả để làm gì nâng bi phí tiền mình không nói bạn có tiền để mua hay không nhưng đơn giản là bạn vào đọc không thích bạn có thể đi ra và tìm 1 topic khác đúng sở thích của không cần phải thở ra những câu chả liên quan kiểu như vậy để người khác không mua giống mình cho đỡ cảm thấy lạc lõng với xu hướng của thế giới \xa0']

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf(train, test):
    tf = TfidfVectorizer(ngram_range=(1, 5))
    tf.fit(train)
    train = tf.transform(train)
    test = tf.transform(test)
    return train, test


def tokenization(train, test, vocab_size, max_len):
    tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
    tokenizer.fit_on_texts(train)

    train = tokenizer.texts_to_sequences(train)
    test = tokenizer.texts_to_sequences(test)
    padded_train = pad_sequences(train, maxlen=max_len, padding='post', truncating='post')
    padded_test = pad_sequences(test, maxlen=max_len, padding='post', truncating='post')
    
    return padded_train, padded_test

# Load dataset

In [11]:
def read_file(path, label):
        fopen = open(file=path, mode='r',encoding='UTF-8')
        X = fopen.read().replace('\xa0', '').split('\n')
        if X[-1] == '': 
            X.pop(-1)
        if label != -1:
            return X[0::2], [label for i in range(len(X[0::2]))]
        else:
            
            label_dict = {'NEG': 0, 'NEU': 1, 'POS': 2}
            return X[0::2], [label_dict[x] for x in X[1::2]]
    
def read_dataset(**kwargs):
    if kwargs['is_train']:
        X_neg, Y_neg = read_file(kwargs['neg_path'], 0)
        X_neu, Y_neu = read_file(kwargs['neu_path'], 1)
        X_pos, Y_pos = read_file(kwargs['pos_path'], 2)
        X = X_neg + X_neu + X_pos
        X = pre_process(X)
        Y = Y_neg + Y_neu + Y_pos
        Y = np.array(Y)
        return X, Y
    else:
        X, Y = read_file(kwargs['path'], -1)
        X = pre_process(X)
        Y = np.array(Y)
        return X, Y

In [12]:
class SA_Dataset():
    def __init__(self, mode, vocab_size=10000, max_len=420, split=False, test_size=0.20588235294):
        self.X_train, self.Y_train = read_dataset(neg_path='/kaggle/input/sa-vi/SA-training_negative.txt', 
                                                  neu_path='/kaggle/input/sa-vi/SA-training_neutral.txt', 
                                                  pos_path='/kaggle/input/sa-vi/SA-training_positive.txt', 
                                                  is_train=True)
        self.X_test, self.Y_test = read_dataset(path='/kaggle/input/sa-vi/test_raw_ANS.txt', is_train=False)
        
        if mode == 'tfidf':
            self.X_train, self.X_test = tfidf(self.X_train, self.X_test)
        elif mode == 'tokenization':
            self.X_train, self.X_test = tokenization(self.X_train, self.X_test, vocab_size, max_len)
        
        if split == True:
            self.X_train, self.X_valid, self.Y_train, self.Y_valid = train_test_split(self.X_train, self.Y_train, 
                                                                                    test_size=test_size, shuffle=True, random_state=42)

# Use Machine Learning model

In [13]:
ds = SA_Dataset(mode='tfidf')

In [14]:
ds.X_train.shape

(5100, 446635)

In [17]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier().fit(ds.X_train, ds.Y_train)
sgd_prediction = sgd.predict(ds.X_test)
print(classification_report(ds.Y_test, sgd_prediction))

              precision    recall  f1-score   support

           0       0.73      0.68      0.70       350
           1       0.67      0.72      0.70       350
           2       0.75      0.75      0.75       350

    accuracy                           0.72      1050
   macro avg       0.72      0.72      0.72      1050
weighted avg       0.72      0.72      0.72      1050



In [19]:
from sklearn.svm import SVC
svc = SVC(C=1000).fit(ds.X_train, ds.Y_train)
svc_prediction = svc.predict(ds.X_test)
print(classification_report(ds.Y_test, svc_prediction))

              precision    recall  f1-score   support

           0       0.73      0.64      0.68       350
           1       0.62      0.77      0.69       350
           2       0.78      0.69      0.73       350

    accuracy                           0.70      1050
   macro avg       0.71      0.70      0.70      1050
weighted avg       0.71      0.70      0.70      1050

